Basically testing out what the backtester needs to do to the logs to make it in correct form

In [2]:
import numpy as np
import pandas as pd


In [3]:
LOG_PATH = "../data/prices_round_1_day_-2.csv"

In [7]:
df = pd.read_csv(LOG_PATH, sep=";")

df.head(6)

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-2,0,AMETHYSTS,9996,1,9995.0,25.0,NaN,NaN,10004,1,10005.0,25.0,NaN,NaN,10000.0,0.0
1,-2,0,STARFRUIT,4997,26,NaN,NaN,NaN,NaN,5003,26,NaN,NaN,NaN,NaN,5000.0,0.0
2,-2,100,AMETHYSTS,10000,5,9995.0,20.0,NaN,NaN,10005,20,NaN,NaN,NaN,NaN,10002.5,0.0
3,-2,100,STARFRUIT,5000,5,4998.0,6.0,4997.0,20.0,5004,20,NaN,NaN,NaN,NaN,5002.0,0.0
4,-2,200,STARFRUIT,4997,1,4996.0,30.0,NaN,NaN,5003,31,NaN,NaN,NaN,NaN,5000.0,0.0
5,-2,200,AMETHYSTS,9996,1,9995.0,30.0,NaN,NaN,10004,1,10005.0,30.0,NaN,NaN,10000.0,0.0


In [23]:
grouped_df = df.groupby("timestamp")

In [37]:
grouped_df.get_group(list(grouped_df.groups)[0])

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-2,0,AMETHYSTS,9996,1,9995.0,25.0,NaN,NaN,10004,1,10005.0,25.0,NaN,NaN,10000.0,0.0
1,-2,0,STARFRUIT,4997,26,NaN,NaN,NaN,NaN,5003,26,NaN,NaN,NaN,NaN,5000.0,0.0


In [48]:
from datamodel import Listing, OrderDepth, TradingState

def stream():
    for name, group in grouped_df:
        listings = {}
        order_depths = {}
        for idx, row in group.iterrows():
            listings[row["product"]] = Listing(row["product"], row["product"], 1)
            order_depths[row["product"]] = OrderDepth()
            for i in (1, 2, 3):
                if not pd.isna(row[f"bid_price_{i}"]):
                    order_depths[row["product"]].buy_orders[str(row[f"bid_price_{i}"])] = row[f"bid_volume_{i}"]
                if not pd.isna(row[f"ask_price_{i}"]):
                    order_depths[row["product"]].sell_orders[str(row[f"ask_price_{i}"])] = row[f"ask_volume_{i}"]
        yield TradingState(
            traderData=None,
            timestamp=name,
            listings=listings,
            order_depths=order_depths,
            market_trades=None,
            observations=None,
            own_trades=None,
            position=None,
        )

s = stream()
val = next(s)

In [52]:
val.order_depths["AMETHYSTS"].sell_orders

{'10004': 1, '10005.0': 25.0}